In [2]:
import pandas as pd
import json
import os
import hana_ml
from hana_ml import ConnectionContext

print(pd.__version__)
print(hana_ml.__version__)

2.2.0
2.19.24022101


In [3]:
#Connect HANA Table
with open(os.path.join(os.getcwd(), 'C:\workspace\hands-on\HANA-configure-jinwook.json')) as f:
    hana_env_c = json.load(f)
    port_c = hana_env_c['port']
    user_c = hana_env_c['user']
    host_c = hana_env_c['host']
    pwd_c = hana_env_c['pwd']

cc = ConnectionContext(address=host_c, port=port_c, user=user_c, password=pwd_c, encrypt=True)
cursor = cc.connection.cursor()
cursor.execute("""SET SCHEMA GEN_AI""")

#Check hana_ml version
print(cc.hana_version())
#Check current schema
print(cc.get_current_schema())


4.00.000.00.1713874676 (CE2024.14)
GEN_AI


In [5]:
data = pd.read_csv('C:\workspace\hands-on\Data\dataset.csv', index_col=False)

data['BlockID'] = range(1, len(data) + 1)
print(data.columns)

Index(['MaterialNumber', 'MaterialName', 'PurchasingGroupDescription',
       'PurchasingGroup', 'MaterialDescription', 'Total_stock',
       'Unrestricted_use_stock', 'Blocked_stock', 'Block_reason',
       'Blocked_date', 'Solution_date', 'Solution',
       'PurchasingGroupDescription_KR', 'MaterialDescription_KR',
       'Solution_KR', 'Block_reason_KR', 'BlockID'],
      dtype='object')


In [6]:
data1 = data[['BlockID','MaterialNumber', 'MaterialName',
'MaterialDescription',
'MaterialDescription_KR','PurchasingGroup','PurchasingGroupDescription',
'PurchasingGroupDescription_KR','Total_stock','Unrestricted_use_stock',
'Blocked_stock', 'Blocked_date',
'Block_reason','Block_reason_KR','Solution', 'Solution_KR',
'Solution_date' ]]

data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   BlockID                        10 non-null     int64 
 1   MaterialNumber                 10 non-null     int64 
 2   MaterialName                   10 non-null     object
 3   MaterialDescription            10 non-null     object
 4   MaterialDescription_KR         10 non-null     object
 5   PurchasingGroup                10 non-null     object
 6   PurchasingGroupDescription     10 non-null     object
 7   PurchasingGroupDescription_KR  10 non-null     object
 8   Total_stock                    10 non-null     int64 
 9   Unrestricted_use_stock         10 non-null     int64 
 10  Blocked_stock                  10 non-null     int64 
 11  Blocked_date                   10 non-null     object
 12  Block_reason                   10 non-null     object
 13  Block_re

In [7]:
cursor = cc.connection.cursor()
sql_command = '''CREATE TABLE "DEMO_BLOCK_JINWOOK" (
"BlockID" BIGINT,
"MaterialNumber" BIGINT,
"MaterialName" NCLOB MEMORY THRESHOLD 0,
"MaterialDescription" NCLOB MEMORY THRESHOLD 0,
"MaterialDescription_KR" NCLOB MEMORY THRESHOLD 0,
"PurchasingGroup" NVARCHAR(100),
"PurchasingGroupDescription" NVARCHAR(5000),
"PurchasingGroupDescription_KR" NVARCHAR(5000),
"Total_stock" BIGINT,
"Unrestricted_use_stock" BIGINT,
"Blocked_stock" BIGINT,
"Blocked_date" DATE,
"Block_reason" NCLOB MEMORY THRESHOLD 0,
"Block_reason_KR" NCLOB MEMORY THRESHOLD 0,
"Solution" NCLOB MEMORY THRESHOLD 0,
"Solution_KR" NCLOB MEMORY THRESHOLD 0,
"Solution_date" DATE)'''

cursor.execute(sql_command)
cursor.close()

ProgrammingError: (288, 'cannot use duplicate table name: DEMO_BLOCK_JINWOOK: line 1 col 14 (at pos 13)')

In [8]:
data2 = data1.copy()
data2['Blocked_date'] = pd.to_datetime(data2['Blocked_date'])
data2['Solution_date'] = pd.to_datetime(data2['Solution_date'])

data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   BlockID                        10 non-null     int64         
 1   MaterialNumber                 10 non-null     int64         
 2   MaterialName                   10 non-null     object        
 3   MaterialDescription            10 non-null     object        
 4   MaterialDescription_KR         10 non-null     object        
 5   PurchasingGroup                10 non-null     object        
 6   PurchasingGroupDescription     10 non-null     object        
 7   PurchasingGroupDescription_KR  10 non-null     object        
 8   Total_stock                    10 non-null     int64         
 9   Unrestricted_use_stock         10 non-null     int64         
 10  Blocked_stock                  10 non-null     int64         
 11  Blocked_date          

C:\Users\cesco\AppData\Local\Temp\ipykernel_24120\3467301026.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data2['Blocked_date'] = pd.to_datetime(data2['Blocked_date'])
C:\Users\cesco\AppData\Local\Temp\ipykernel_24120\3467301026.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data2['Solution_date'] = pd.to_datetime(data2['Solution_date'])


In [9]:
# import dataframe into hana table
from hana_ml.dataframe import create_dataframe_from_pandas
v_hdf = create_dataframe_from_pandas(
    connection_context=cc,
    pandas_df=data2,
    table_name="DEMO_BLOCK_JINWOOK",
    allow_bigint=True,
    append=True,
    force=False)

  0%|          | 0/1 [00:00<?, ?it/s]


ProgrammingError: (270, 'not enough values: :17: line 1 col 74 (at pos 73)')

In [10]:
hdf = cc.sql('''SELECT TOP 10 "BlockID","MaterialNumber",
"MaterialName","Blocked_stock","Block_reason","Blocked_date",
"Solution", "Solution_KR","Solution_date"
FROM DEMO_BLOCK_JINWOOK''')

df_abstract = hdf.collect()
df_abstract.head()

,BlockID,MaterialNumber,MaterialName,Blocked_stock,Block_reason,Blocked_date,Solution,Solution_KR,Solution_date
0,1,59829111,PO√ÑNG,2695,Safety concerns trigger an urgent recall of sp...,2024-01-20,"[{""Instruction_step"": 1, ""Instruction"": ""Ident...","[{""Instruction_step"": 1, ""Instruction_KR"": ""영향...",2024-01-31
1,2,250249,VITTSJ√ñ,34657,Identification of packaging defects that compr...,2024-03-10,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-23
2,3,10364523,KALLVIKEN,2262,Extended customs hold for certain shipments du...,2024-03-10,"[{""Instruction_step"": 1, ""Instruction"": ""Provi...","[{""Instruction_step"": 1, ""Instruction_KR"": ""관세...",2024-05-06
3,4,39213915,KLIMPEN,21111,Critical feedback on packaging quality leading...,2024-06-20,"[{""Instruction_step"": 1, ""Instruction"": ""Gathe...","[{""Instruction_step"": 1, ""Instruction_KR"": ""중요...",2024-06-27
4,5,50391421,NORDLI,23208,Discrepancies in item quantity received by cus...,2024-05-06,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""주문...",2024-05-08


In [18]:
cursor = cc.connection.cursor()
sql_command = '''ALTER TABLE DEMO_BLOCK_JINWOOK ADD (VECTOR_EN
REAL_VECTOR(1536))'''

cursor.execute(sql_command)
cursor.close()


In [31]:
cursor = cc.connection.cursor()
sql_command = '''ALTER TABLE DEMO_BLOCK_JINWOOK ADD (VECTOR_KR
REAL_VECTOR(1536))'''

cursor.execute(sql_command)
cursor.close()

In [23]:
# get embedding
from gen_ai_hub.proxy.native.openai import embeddings
def get_embedding(input, model="dc872f9eef04c31a") -> str:
    response = embeddings.create(
        deployment_id=model,
        input=input
    )   
    return response.data[0].embedding

In [24]:
get_embedding("Nice to Meet you!")

[-0.0013131164014339447,
 -0.01177543867379427,
 0.008583675138652325,
 -0.031211109831929207,
 -0.015146932564675808,
 0.024443332105875015,
 -0.0064826891757547855,
 0.0010535919573158026,
 -0.025100277736783028,
 -0.014180107042193413,
 0.016026992350816727,
 -0.005664605647325516,
 0.0008382252999581397,
 -0.014242083765566349,
 0.010151667520403862,
 -0.01103172730654478,
 0.025137463584542274,
 -0.01571711152791977,
 0.012940587475895882,
 -0.0018515328411012888,
 -0.006792569067329168,
 0.00124804163351655,
 0.002801314927637577,
 -0.00425155321136117,
 -0.007982508279383183,
 -0.0013224127469584346,
 -0.013101724907755852,
 -0.018270522356033325,
 -0.0035760148894041777,
 -0.007406131364405155,
 0.018493637442588806,
 0.0036968679632991552,
 -0.005518962163478136,
 -0.0040346370078623295,
 0.002497632522135973,
 -0.030888834968209267,
 -0.011533732526004314,
 -0.0025580590590834618,
 0.008348166011273861,
 -0.010331397876143456,
 0.019510043784976006,
 -0.00564601318910718,
 0.

In [25]:
data2['Context'] = 'Block reason:' + data2['Block_reason'] \
                    + '\n Material Number:' + data2['MaterialNumber'].astype(str) \
                    + '\n Material Name:' + data2['MaterialName'] \
                    + '\n Material Description:' + data2['MaterialDescription'] \
                    + '\n Purchasing Group:' + data2['PurchasingGroup'] \
                    + '\n Purchasing Group Description:' + data2['PurchasingGroupDescription']

data2['Context_KR'] = '블럭 원인:' + data2['Block_reason_KR'] \
                    + '\n 자재 번호:' + data2['MaterialNumber'].astype(str) \
                    + '\n 자재명:' + data2['MaterialName'] \
                    + '\n 자재 설명:' + data2['MaterialDescription_KR'] \
                    + '\n 자재 그룹:' + data2['PurchasingGroup'] \
                    + '\n 구매 그룹 설명:' + data2['PurchasingGroupDescription_KR']

In [26]:
#generate embeddings from the text
rows = []
rows_kr = []

for index, row in data2.iterrows():
    text = row['Context']
    #print(text)
    try:
        text_vector = get_embedding(input=text, model="dc872f9eef04c31a")
        myrow = (str(text_vector), row['BlockID'])
        rows.append(myrow)
    except Exception as e:
        print(e)


for index, row in data2.iterrows():
    text_kr = row['Context_KR']
    #print(text)
    try:
        text_vector_kr = get_embedding(input=text_kr, model="dc872f9eef04c31a")
        myrow_kr = (str(text_vector_kr), row['BlockID'])
        rows_kr.append(myrow_kr)
    except Exception as e:
        print(e)

In [27]:
print(rows[0])
print(rows_kr[0])

('[0.01642700284719467, -0.014308784157037735, -0.014085434377193451, -0.01176548097282648, -0.027767399325966835, 0.03501545265316963, -0.042681388556957245, -0.016830474138259888, -0.0035393706057220697, -0.013278493657708168, 0.007557862438261509, -0.0005421631503850222, -0.025087203830480576, 0.012039263732731342, -0.03717690333724022, 0.021340694278478622, 0.007579477038234472, 0.01858845166862011, 0.0004361171158961952, -0.0018984716152772307, -0.0012923656031489372, 0.03270990774035454, -0.018660498782992363, 0.015115724876523018, -0.010202033445239067, 0.002907147165387869, 0.048531707376241684, -0.0341796949505806, -0.0170898474752903, -0.00012856110697612166, 0.01982768066227436, -0.043200135231018066, -0.025087203830480576, -0.01553360465914011, -0.0116069745272398, 0.0005980005371384323, -0.016196448355913162, -0.0016408991068601608, 0.0025505083613097668, -0.018703728914260864, 0.027796218171715736, 0.019251294434070587, 0.00598721019923687, 0.005335173569619656, 0.0086962

In [29]:
# bulk update
# cc.connection.setautocommit(False)
cursor = cc.connection.cursor()
sql = 'UPDATE DEMO_BLOCK_JINWOOK SET VECTOR_EN = TO_REAL_VECTOR(?) WHERE "BlockID" = ?'
try:
    cursor.executemany(sql, rows)
except Exception as e:
    cc.connection.rollback()
    print("An error occurred:", e)
try:
    cc.connection.commit()
finally:
    cursor.close()
cc.connection.setautocommit(True)

In [32]:
# bulk_kr update
# cc.connection.setautocommit(False)
cursor = cc.connection.cursor()
sql = 'UPDATE DEMO_BLOCK_JINWOOK SET VECTOR_KR = TO_REAL_VECTOR(?) WHERE "BlockID" = ?'
try:
    cursor.executemany(sql, rows_kr)
except Exception as e:
    cc.connection.rollback()
    print("An error occurred:", e)
try:
    cc.connection.commit()
finally:
    cursor.close()
cc.connection.setautocommit(True)

In [11]:
# Check Updates
cc = ConnectionContext(address=host_c, port=port_c, user= user_c, password = pwd_c, encrypt=True)
cursor = cc.connection.cursor()
cursor.execute("""SET SCHEMA GEN_AI""")

hdf = cc.sql('''SELECT TOP 5 "BlockID","MaterialNumber", "MaterialName",
"Blocked_stock","Block_reason","Blocked_date", "Solution",
"Solution_KR","Solution_date", "VECTOR_KR"
FROM DEMO_BLOCK_JINWOOK''')

df_abstract = hdf.collect()
df_abstract.head(5)

,BlockID,MaterialNumber,MaterialName,Blocked_stock,Block_reason,Blocked_date,Solution,Solution_KR,Solution_date,VECTOR_KR
0,1,59829111,PO√ÑNG,2695,Safety concerns trigger an urgent recall of sp...,2024-01-20,"[{""Instruction_step"": 1, ""Instruction"": ""Ident...","[{""Instruction_step"": 1, ""Instruction_KR"": ""영향...",2024-01-31,"[0, 6, 0, 0, 178, 206, 196, 187, 195, 47, 200,..."
1,2,250249,VITTSJ√ñ,34657,Identification of packaging defects that compr...,2024-03-10,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-23,"[0, 6, 0, 0, 86, 57, 26, 188, 244, 158, 76, 18..."
2,3,10364523,KALLVIKEN,2262,Extended customs hold for certain shipments du...,2024-03-10,"[{""Instruction_step"": 1, ""Instruction"": ""Provi...","[{""Instruction_step"": 1, ""Instruction_KR"": ""관세...",2024-05-06,"[0, 6, 0, 0, 114, 28, 15, 59, 0, 12, 233, 188,..."
3,4,39213915,KLIMPEN,21111,Critical feedback on packaging quality leading...,2024-06-20,"[{""Instruction_step"": 1, ""Instruction"": ""Gathe...","[{""Instruction_step"": 1, ""Instruction_KR"": ""중요...",2024-06-27,"[0, 6, 0, 0, 69, 123, 19, 188, 150, 9, 225, 18..."
4,5,50391421,NORDLI,23208,Discrepancies in item quantity received by cus...,2024-05-06,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""주문...",2024-05-08,"[0, 6, 0, 0, 13, 251, 61, 188, 26, 157, 177, 1..."
